# 전처리 종합실습 평가

#### 목표
1. 간략히 가설수립을 합니다.
2. 데이터를 탐색합니다.(역시 간략히)
3. 가설로 도출된 요인들을 추가변수로 생성합니다.
4. 모델링을 위한 전처리를 수행합니다.
5. 준비된 모델링 코드로 성능을 확인합니다.

#### 규칙
1. 개념에 대한 질문, 문제의 이해를 위한 질문에 대해 답변을 드립니다.
2. 오류에 대해서는 해결 방안과 힌트를 제공해 드립니다.
3. 참조 가능한 자료는 동료들의 소스코드를 제외한 모든 자료를 참조할 수 있습니다. 

# 1단계: 비즈니스이해~ Labeling까지



## 1. 비즈니스 이해

문제 정의 : 
* 우리는 고객이 이탈할 것인지 사전에 예측하는 모델을 만들고자 합니다.
* 이탈할 고객을 미리 예측할 수 있다면, 프로모션을 통해 이탈을 예방할 수 있습니다.
* 그렇다면 고객의 이탈에 영향을 주는 요인은 무엇일까요?


다음과 같이 구체적으로 문제를 정의하고자 한다.
* 2016년12월말 기준으로
* 이후 3개월 방문(구매) 안한 사람을 이탈로 보자.
* 관심있는 고객은 
    - 2015~2016년 신규 가입 고객 이면서,
    - 2016년 하반기에 한번 이상 방문한 고객


### 문제1 :  
아래 셀에 이탈여부와 관련한 요인을 최소한 5가지 이상을 도출해 봅시다.  
이 질문에 대한 답은 3.데이터준비단계를 수행하면서 다시 여기로 돌아와서 수정하거나 추가해야 할 수도 있습니다.


예시1) 2016년에 4번이상 방문하고 10000만원 이상구매한 고객은 이탈하지 않을 것이다.  
예시2) 남성고객이면서, 30대 고객은 고객은 이탈할 것이다.   
    
    
    1) 구매채널이 다양한 20~30대 고객은 이탈할 것이다.
    2) 
    3)
    4)
    5)
    6)
    7)
    8)
    9)
    10)
    


## 2.데이터 이해

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [177]:
customers = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/customers.csv'
                 , sep=',', skipinitialspace=True, encoding='CP949')  
customers.head()

,CustomerID,RegisterDate,Address,Gender,BirthYear,Addr1,Addr2
0,c328222,2014-09-25,강원 원주시 늘품로,F,1960,강원도,원주시
1,c281448,2013-06-18,강원 원주시 치악로,F,1974,강원도,원주시
2,c038336,2003-10-10,강원 춘천시 서부대성로,F,1968,강원도,춘천시
3,c084237,2007-03-09,강원도 강릉시 연곡면 황어대길,F,1982,강원도,강릉시
4,c162600,2010-06-14,강원도 속초시 농공단지길,F,1978,강원도,속초시


In [178]:
products = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv'
                 , sep=',', skipinitialspace=True, encoding='CP949')  
products.head()

,ProductID,ProductName,Category,SubCategory,CategoryOrd
0,p1052661,새우깡,간식,과자,3
1,p1054261,고구마스틱,간식,과자,3
2,p1097821,짱구,간식,과자,3
3,p1097831,감자칩,간식,과자,3
4,p1119071,뿌셔뿌셔,간식,과자,3


In [179]:
sales = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv'
                 , sep=',', skipinitialspace=True, encoding='CP949')  
sales.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID
0,107,2,2016-01-02,p1036481,2,2100,c150417
1,69,1,2016-01-02,p1152861,1,1091,c212716
2,69,7,2016-01-02,p1013161,1,2600,c212716
3,69,8,2016-01-02,p1005771,1,1650,c212716
4,69,11,2016-01-02,p1089531,1,2600,c212716


In [180]:
sales['OrderDate'] = pd.to_datetime(sales['OrderDate']) 
customers['RegisterDate'] = pd.to_datetime(customers['RegisterDate']) 

**데이터 탐색은 3.데이터 준비 단계 후에 진행해 봅시다.**

###  문제 2. 데이터 살펴보기

2-1) 2016년 월별 매출액은?

In [181]:
sales['year'] = sales['OrderDate'].dt.year
sales['month'] = sales['OrderDate'].dt.month
sales['day'] = sales['OrderDate'].dt.day
sales.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,year,month,day
0,107,2,2016-01-02,p1036481,2,2100,c150417,2016,1,2
1,69,1,2016-01-02,p1152861,1,1091,c212716,2016,1,2
2,69,7,2016-01-02,p1013161,1,2600,c212716,2016,1,2
3,69,8,2016-01-02,p1005771,1,1650,c212716,2016,1,2
4,69,11,2016-01-02,p1089531,1,2600,c212716,2016,1,2


In [182]:
sales_2016 = sales.groupby(by = ['year', 'month'], as_index=False)[['Amt']].sum()
sales_2016.head(12)

,year,month,Amt
0,2016,1,14209990
1,2016,2,14367451
2,2016,3,17111120
3,2016,4,15129917
4,2016,5,13976941
5,2016,6,13113610
6,2016,7,10717743
7,2016,8,10176702
8,2016,9,10496653
9,2016,10,10003648


2-2) 총 가입고객수는?

In [183]:
customers['CustomerID'].duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2238    False
2239    False
2240    False
2241    False
2242    False
Name: CustomerID, Length: 2243, dtype: bool

In [184]:
total_customers = customers['CustomerID'].count()
total_customers

2243

2-3) 2015~2016에 가입한 고객은 총 몇명인가?

In [185]:
customers.head()

,CustomerID,RegisterDate,Address,Gender,BirthYear,Addr1,Addr2
0,c328222,2014-09-25,강원 원주시 늘품로,F,1960,강원도,원주시
1,c281448,2013-06-18,강원 원주시 치악로,F,1974,강원도,원주시
2,c038336,2003-10-10,강원 춘천시 서부대성로,F,1968,강원도,춘천시
3,c084237,2007-03-09,강원도 강릉시 연곡면 황어대길,F,1982,강원도,강릉시
4,c162600,2010-06-14,강원도 속초시 농공단지길,F,1978,강원도,속초시


In [186]:
customers1516 = customers[customers['RegisterDate'].between('2015-01-01','2016-12-31')].copy()
customers1516['CustomerID'].count()

485

2-4) 2016년 하반기에 구매한 적이 있는 고객의 수는?

In [187]:
sales.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,year,month,day
0,107,2,2016-01-02,p1036481,2,2100,c150417,2016,1,2
1,69,1,2016-01-02,p1152861,1,1091,c212716,2016,1,2
2,69,7,2016-01-02,p1013161,1,2600,c212716,2016,1,2
3,69,8,2016-01-02,p1005771,1,1650,c212716,2016,1,2
4,69,11,2016-01-02,p1089531,1,2600,c212716,2016,1,2


In [188]:
sales16 = sales[sales['OrderDate'].between('2016-07-01','2016-12-31')].copy()
sales16 = sales16['CustomerID']
sales16.drop_duplicates(keep='first', inplace=True)
sales16.count()

1711

In [ ]:
# 1번가설 : 구매 채널이 다양한 20~30대 고객은 이탈할 것이다.                       
# 




In [ ]:
# 2번가설 :   





## 3. 데이터 준비(Preprocessing)

### 문제 3. Labeling

우리는 고객이 이탈할 것인지 사전에 예측하는 모델을 만들고자 한다.  
그렇다면 고객의 이탈을 어떻게 정의할 것인가?  

* 2016년12월말 기준
* 이후 3개월 방문 안한 사람을 이탈로 보자.
* 관심있는 고객은 
    - 2015~2016년 신규 가입 고객 이면서,
    - 2016년 하반기에 한번 이상 방문한 고객


3-1) 고객ID 조회 : 
    - 2016년 하반기에 한번 이상 방문한 고객 중 2015~2016년 신규 가입 고객을 조회하시오.
    
    * .between(     )
    * .drop_duplications()

In [189]:
sales16 = sales[sales['OrderDate'].between('2016-07-01','2016-12-31')].copy()
sales16.drop_duplicates(subset='CustomerID', inplace=True)
sales16

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,year,month,day
29519,89,1,2016-07-01,p1030071,1,2450,c127482,2016,7,1
29522,101,1,2016-07-01,p1011291,1,273,c217179,2016,7,1
29523,103,1,2016-07-01,p1175481,1,1300,c222561,2016,7,1
29526,107,9,2016-07-01,p1011291,2,546,c333911,2016,7,1
29527,111,3,2016-07-01,p1030071,1,2450,c373441,2016,7,1
...,...,...,...,...,...,...,...,...,...,...
55591,37,5,2016-12-29,p1175481,1,1150,c153641,2016,12,29
55627,97,2,2016-12-30,p1005771,2,3700,c323093,2016,12,30
55670,25,2,2016-12-30,p1005891,1,1950,c313636,2016,12,30
55836,67,5,2016-12-31,p1207281,1,3300,c188728,2016,12,31


In [190]:
customers1516 = customers[customers['RegisterDate'].between('2015-01-01','2016-12-31')].copy()
customers1516

,CustomerID,RegisterDate,Address,Gender,BirthYear,Addr1,Addr2
12,c354310,2015-07-16,강원도 원주시 무실로,F,1964,강원도,원주시
16,c390828,2016-10-15,강원도 원주시 시청로,F,1984,강원도,원주시
28,c386399,2016-08-18,강원도 원주시 호저면 신무로,F,1972,강원도,원주시
30,c367116,2015-12-29,강원도 원주시 호저면 영산길,F,1970,강원도,원주시
33,c345766,2015-04-12,강원도 춘천시 서부대성로,M,1980,강원도,춘천시
...,...,...,...,...,...,...,...
2206,c368015,2016-01-09,인천광역시 연수구 해돋이로,F,1982,인천광역시,연수구
2214,c366415,2015-12-18,인천광역시 옹진군 북도면 시도로,F,1978,인천광역시,옹진군
2218,c341674,2015-02-28,인천광역시 중구 신도시북로88번길,M,1973,인천광역시,중구
2228,c352708,2015-06-26,충남 당진시 무수동옛길,F,1980,충청남도,당진시


In [191]:
good_customers = pd.merge(sales16, customers1516, on='CustomerID', how='inner')
good_customers = good_customers[['CustomerID']]
good_customers

,CustomerID
0,c373441
1,c337831
2,c337999
3,c341674
4,c349783
...,...
349,c396022
350,c396059
351,c392672
352,c345428


3-2) 3-1)의 결과에 해당하는 고객ID 별로 2017년 1~3월 방문 여부(이탈여부) 데이터프레임 만들기
    - 고객ID
    - 이탈여부 : (주의)변수 이름을 반드시 Churn 으로 만들어 주세요. 1이 이탈, 0은 잔류 입니다.


In [192]:
# 2017년 1~3월 방문 고객
sales17=sales[sales['OrderDate'].between('2017-01-01','2017-03-31')].copy()
sales17.drop_duplicates(subset='CustomerID', inplace=True)
sales17 = sales17[['CustomerID']]
sales17['visit'] = 0
sales17

,CustomerID,visit
55976,c271068,0
55982,c146132,0
55983,c226242,0
55984,c140658,0
55985,c328839,0
...,...,...
69871,c400253,0
69883,c044830,0
69888,c346882,0
69904,c095005,0


In [389]:
customer_churn = pd.merge(good_customers,sales17, on='CustomerID', how='left')
customer_churn['visit'].loc[customer_churn['visit'].isna()] = 1
customer_churn.rename(columns={'visit':'Churn'}, inplace=True)
customer_churn

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,CustomerID,Churn
0,c373441,0.0
1,c337831,1.0
2,c337999,0.0
3,c341674,0.0
4,c349783,0.0
...,...,...
349,c396022,0.0
350,c396059,0.0
351,c392672,1.0
352,c345428,1.0


In [390]:
customer_churn.Churn.value_counts()

0.0    226
1.0    128
Name: Churn, dtype: int64

# 여기까지 수행 후 1차 제출합니다.
여기까지 수행한 후 파일이름 뒤에 **_자신의이름**  을 붙여서 제출합니다.  _

같이 풀이후, 정답을 제공받아서 다음을 진행합니다.


# 2단계 : 변수 추가 ~ 모델링까지!

### 문제 4. 변수 추가하기

문제 1에서 도출된 요인들을 변수로 만들기  
**3-2) 문제 결과에 변수들을 추가합니다.**

아래 문제들을 풀고, 여러분의 변수들을 추가해 봅시다.

현재를 2016년12월31일로 간주합니다.

4-1) 성별, 나이 추가하기  

In [391]:
customer_churn = pd.merge(customer_churn, customers[['CustomerID','Gender','BirthYear']], on='CustomerID', how='inner')
customer_churn

,CustomerID,Churn,Gender,BirthYear
0,c373441,0.0,F,1977
1,c337831,1.0,F,1984
2,c337999,0.0,F,1977
3,c341674,0.0,M,1973
4,c349783,0.0,F,1973
...,...,...,...,...
349,c396022,0.0,F,1983
350,c396059,0.0,F,1962
351,c392672,1.0,F,1977
352,c345428,1.0,F,1960


In [392]:
customer_churn['Age'] = 2016 - customer_churn['BirthYear']
customer_churn

,CustomerID,Churn,Gender,BirthYear,Age
0,c373441,0.0,F,1977,39
1,c337831,1.0,F,1984,32
2,c337999,0.0,F,1977,39
3,c341674,0.0,M,1973,43
4,c349783,0.0,F,1973,43
...,...,...,...,...,...
349,c396022,0.0,F,1983,33
350,c396059,0.0,F,1962,54
351,c392672,1.0,F,1977,39
352,c345428,1.0,F,1960,56


4-2) 최근 3개월 구매액, 방문횟수

In [393]:
sales.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,year,month,day
0,107,2,2016-01-02,p1036481,2,2100,c150417,2016,1,2
1,69,1,2016-01-02,p1152861,1,1091,c212716,2016,1,2
2,69,7,2016-01-02,p1013161,1,2600,c212716,2016,1,2
3,69,8,2016-01-02,p1005771,1,1650,c212716,2016,1,2
4,69,11,2016-01-02,p1089531,1,2600,c212716,2016,1,2


In [394]:
sales_recent3 = sales[sales['OrderDate'].between('2016-10-01','2016-12-31')].copy()
sales_recent3_Amy = sales_recent3.groupby(by='CustomerID', as_index=False)[['Amt']].sum()
sales_recent3_Count = sales_recent3.groupby(by=['CustomerID'], as_index=False)[['OrderID']].count()
sales_recent3_Count

,CustomerID,OrderID
0,c017487,18
1,c017503,5
2,c017517,21
3,c017522,11
4,c017526,4
...,...,...
1377,c395401,3
1378,c395638,7
1379,c395673,1
1380,c396022,3


In [395]:
customer_churn = pd.merge(customer_churn, sales_recent3_Amy, on='CustomerID', how='left')
customer_churn = pd.merge(customer_churn, sales_recent3_Count, on='CustomerID', how='left')
customer_churn

,CustomerID,Churn,Gender,BirthYear,Age,Amt,OrderID
0,c373441,0.0,F,1977,39,17491.0,5.0
1,c337831,1.0,F,1984,32,NaN,NaN
2,c337999,0.0,F,1977,39,93637.0,43.0
3,c341674,0.0,M,1973,43,86163.0,37.0
4,c349783,0.0,F,1973,43,7800.0,2.0
...,...,...,...,...,...,...,...
349,c396022,0.0,F,1983,33,7955.0,3.0
350,c396059,0.0,F,1962,54,10320.0,2.0
351,c392672,1.0,F,1977,39,2182.0,1.0
352,c345428,1.0,F,1960,56,3727.0,1.0


4-3) 최근 3개월 카테고리(category)별 구매액

In [396]:
Products_recent3 = pd.merge(sales, products[['ProductID','CategoryOrd']], on='ProductID', how='inner')
Products_recent3 = Products_recent3[Products_recent3['OrderDate'].between('2016-10-01','2016-12-31')].copy()
Products_recent3

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,year,month,day,CategoryOrd
367,65,8,2016-10-02,p1036481,1,1050,c290510,2016,10,2,1
368,83,8,2016-10-02,p1036481,2,2100,c110521,2016,10,2,1
369,55,19,2016-10-02,p1036481,1,1050,c099967,2016,10,2,1
370,115,6,2016-10-03,p1036481,1,1050,c314144,2016,10,3,1
371,103,1,2016-10-03,p1036481,1,945,c219130,2016,10,3,1
...,...,...,...,...,...,...,...,...,...,...,...
69572,63,8,2016-12-31,p1299491,1,1850,c177887,2016,12,31,5
69573,123,13,2016-12-31,p1299491,1,1850,c347991,2016,12,31,5
69574,149,4,2016-12-31,p1299491,1,1850,c222561,2016,12,31,5
69575,87,8,2016-12-31,p1299491,1,1850,c038319,2016,12,31,5


In [397]:
Products_recent3 = Products_recent3.groupby(by=['CustomerID','CategoryOrd'], as_index=False)[['Amt']].sum()

In [398]:
Products_recent3_wide = Products_recent3.pivot(index='CustomerID', columns='CategoryOrd', values='Amt')
Products_recent3_wide.columns = ['cat_1','cat_2','cat_3','cat_4','cat_5']
Products_recent3_wide

,cat_1,cat_2,cat_3,cat_4,cat_5
CustomerID,,,,,
c017487,8950.0,NaN,9546.0,11960.0,13750.0
c017503,NaN,17600.0,NaN,2750.0,4050.0
c017517,6500.0,8000.0,12317.0,15704.0,15450.0
c017522,13477.0,NaN,NaN,NaN,8550.0
c017526,5800.0,NaN,NaN,NaN,3950.0
...,...,...,...,...,...
c395401,NaN,NaN,864.0,8250.0,NaN
c395638,5077.0,8370.0,1455.0,4600.0,5300.0
c395673,NaN,4200.0,NaN,NaN,NaN


In [399]:
customer_churn = pd.merge(customer_churn, Products_recent3_wide, on='CustomerID', how='left')
customer_churn

,CustomerID,Churn,Gender,BirthYear,Age,Amt,OrderID,cat_1,cat_2,cat_3,cat_4,cat_5
0,c373441,0.0,F,1977,39,17491.0,5.0,800.0,13100.0,1091.0,NaN,2500.0
1,c337831,1.0,F,1984,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c337999,0.0,F,1977,39,93637.0,43.0,11400.0,NaN,6091.0,53446.0,22700.0
3,c341674,0.0,M,1973,43,86163.0,37.0,11850.0,16720.0,6136.0,19927.0,31530.0
4,c349783,0.0,F,1973,43,7800.0,2.0,7800.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
349,c396022,0.0,F,1983,33,7955.0,3.0,NaN,NaN,1455.0,4150.0,2350.0
350,c396059,0.0,F,1962,54,10320.0,2.0,1950.0,8370.0,NaN,NaN,NaN
351,c392672,1.0,F,1977,39,2182.0,1.0,NaN,NaN,2182.0,NaN,NaN
352,c345428,1.0,F,1960,56,3727.0,1.0,NaN,NaN,NaN,3727.0,NaN


4-4) 가입 개월수  
일수 : 날짜1 - 날짜2  
개월수 : 일수 / np.timedelta64(1,'M'))

In [400]:
customers['join_Month'] = (pd.to_datetime('2016-12-31') - customers['RegisterDate']) / np.timedelta64(1,'M')

In [401]:
customers.head()

,CustomerID,RegisterDate,Address,Gender,BirthYear,Addr1,Addr2,join_Month
0,c328222,2014-09-25,강원 원주시 늘품로,F,1960,강원도,원주시,27.203844
1,c281448,2013-06-18,강원 원주시 치악로,F,1974,강원도,원주시,42.448510
2,c038336,2003-10-10,강원 춘천시 서부대성로,F,1968,강원도,춘천시,158.721945
3,c084237,2007-03-09,강원도 강릉시 연곡면 황어대길,F,1982,강원도,강릉시,117.784759
4,c162600,2010-06-14,강원도 속초시 농공단지길,F,1978,강원도,속초시,78.588883


In [402]:
customer_churn = pd.merge(customer_churn, customers[['CustomerID','join_Month']], on='CustomerID', how='left')
customer_churn

,CustomerID,Churn,Gender,BirthYear,Age,Amt,OrderID,cat_1,cat_2,cat_3,cat_4,cat_5,join_Month
0,c373441,0.0,F,1977,39,17491.0,5.0,800.0,13100.0,1091.0,NaN,2500.0,9.495062
1,c337831,1.0,F,1984,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.721226
2,c337999,0.0,F,1977,39,93637.0,43.0,11400.0,NaN,6091.0,53446.0,22700.0,23.622662
3,c341674,0.0,M,1973,43,86163.0,37.0,11850.0,16720.0,6136.0,19927.0,31530.0,22.078482
4,c349783,0.0,F,1973,43,7800.0,2.0,7800.0,NaN,NaN,NaN,NaN,19.220107
...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,c396022,0.0,F,1983,33,7955.0,3.0,NaN,NaN,1455.0,4150.0,2350.0,0.262839
350,c396059,0.0,F,1962,54,10320.0,2.0,1950.0,8370.0,NaN,NaN,NaN,0.229984
351,c392672,1.0,F,1977,39,2182.0,1.0,NaN,NaN,2182.0,NaN,NaN,1.839874
352,c345428,1.0,F,1960,56,3727.0,1.0,NaN,NaN,NaN,3727.0,NaN,20.797142


4-5) 그외 여러분이 도출한 가설에 포함된 요인을 추가 변수로 5개 이상 만드시오.

4-6) 위에서 만든 데이터프레임을 customer_churn 이름으로 만들기  
**이 이름으로 만들어야 모델링 코드가 동작합니다.**

### 문제 5. 데이터를 살펴봅시다.
다시 데이터 이해로 돌아갑니다.  
데이터 준비에서 정리한 데이터셋을 가지고, 가설이 참인지 거짓인지 살펴봅니다.  
이때, 기초통계량과 차트를 이용하게 됩니다. 그리고 이 단계를 특별히 **'탐색적 데이터 분석(EDA)'** 이라고 부릅니다.  
여기서는 몇가지 방법을 가지고 살펴보겠습니다.

5-1) 데이터셋에서 이탈한 고객과 이탈하지 않은 고객의 비율은?

5-2) 성별 별로 이탈율를 조회하시오.

5-3) 나이의 분포를 히스토그램으로 그려봅시다.

5-4) 나이와 이탈여부와의 관계를 차트로 그리려면 코드는 다음과 같습니다.
다음 코드를 수행해 봅시다.

In [ ]:
sns.FacetGrid(customer_churn, hue='Churn', size = 5)\
    .map(sns.distplot, 'Age', kde=True, hist=False)
plt.show()

다음의 조건에 대해서 차트를 그리시오,
* 최근3개월 구매액과 이탈여부
* 최근3개월 방문횟수와 이탈여부
* 가입개월수와 이탈여부

## 4.모델링

모델링을 위한 전처리
    * Dummry Variable
    * Scaling
   

### 문제 6. Dummy variable만들기


범주형 변수들을 가변수화 합니다.(가변수화 이후 기존 변수는 제거합니다.)

In [403]:
customer_churn.head()

,CustomerID,Churn,Gender,BirthYear,Age,Amt,OrderID,cat_1,cat_2,cat_3,cat_4,cat_5,join_Month
0,c373441,0.0,F,1977,39,17491.0,5.0,800.0,13100.0,1091.0,NaN,2500.0,9.495062
1,c337831,1.0,F,1984,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.721226
2,c337999,0.0,F,1977,39,93637.0,43.0,11400.0,NaN,6091.0,53446.0,22700.0,23.622662
3,c341674,0.0,M,1973,43,86163.0,37.0,11850.0,16720.0,6136.0,19927.0,31530.0,22.078482
4,c349783,0.0,F,1973,43,7800.0,2.0,7800.0,NaN,NaN,NaN,NaN,19.220107


In [404]:
Gender_dummy = pd.get_dummies(customer_churn['Gender'])
Gender_dummy

,F,M
0,1,0
1,1,0
2,1,0
3,0,1
4,1,0
...,...,...
349,1,0
350,1,0
351,1,0
352,1,0


In [405]:
customer_churn = pd.concat([customer_churn, Gender_dummy], axis=1, join='inner')

In [406]:
customer_churn.fillna(0, inplace=True)
customer_churn

,CustomerID,Churn,Gender,BirthYear,Age,Amt,OrderID,cat_1,cat_2,cat_3,cat_4,cat_5,join_Month,F,M
0,c373441,0.0,F,1977,39,17491.0,5.0,800.0,13100.0,1091.0,0.0,2500.0,9.495062,1,0
1,c337831,1.0,F,1984,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.721226,1,0
2,c337999,0.0,F,1977,39,93637.0,43.0,11400.0,0.0,6091.0,53446.0,22700.0,23.622662,1,0
3,c341674,0.0,M,1973,43,86163.0,37.0,11850.0,16720.0,6136.0,19927.0,31530.0,22.078482,0,1
4,c349783,0.0,F,1973,43,7800.0,2.0,7800.0,0.0,0.0,0.0,0.0,19.220107,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,c396022,0.0,F,1983,33,7955.0,3.0,0.0,0.0,1455.0,4150.0,2350.0,0.262839,1,0
350,c396059,0.0,F,1962,54,10320.0,2.0,1950.0,8370.0,0.0,0.0,0.0,0.229984,1,0
351,c392672,1.0,F,1977,39,2182.0,1.0,0.0,0.0,2182.0,0.0,0.0,1.839874,1,0
352,c345428,1.0,F,1960,56,3727.0,1.0,0.0,0.0,0.0,3727.0,0.0,20.797142,1,0


불필요한 변수를 제거합니다.

In [407]:
customer_churn.drop(['CustomerID','BirthYear','Gender'], axis=1, inplace=True)
customer_churn

,Churn,Age,Amt,OrderID,cat_1,cat_2,cat_3,cat_4,cat_5,join_Month,F,M
0,0.0,39,17491.0,5.0,800.0,13100.0,1091.0,0.0,2500.0,9.495062,1,0
1,1.0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.721226,1,0
2,0.0,39,93637.0,43.0,11400.0,0.0,6091.0,53446.0,22700.0,23.622662,1,0
3,0.0,43,86163.0,37.0,11850.0,16720.0,6136.0,19927.0,31530.0,22.078482,0,1
4,0.0,43,7800.0,2.0,7800.0,0.0,0.0,0.0,0.0,19.220107,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
349,0.0,33,7955.0,3.0,0.0,0.0,1455.0,4150.0,2350.0,0.262839,1,0
350,0.0,54,10320.0,2.0,1950.0,8370.0,0.0,0.0,0.0,0.229984,1,0
351,1.0,39,2182.0,1.0,0.0,0.0,2182.0,0.0,0.0,1.839874,1,0
352,1.0,56,3727.0,1.0,0.0,0.0,0.0,3727.0,0.0,20.797142,1,0


In [408]:
customer_churn.fillna(0, inplace=True)

In [409]:
customer_churn

,Churn,Age,Amt,OrderID,cat_1,cat_2,cat_3,cat_4,cat_5,join_Month,F,M
0,0.0,39,17491.0,5.0,800.0,13100.0,1091.0,0.0,2500.0,9.495062,1,0
1,1.0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.721226,1,0
2,0.0,39,93637.0,43.0,11400.0,0.0,6091.0,53446.0,22700.0,23.622662,1,0
3,0.0,43,86163.0,37.0,11850.0,16720.0,6136.0,19927.0,31530.0,22.078482,0,1
4,0.0,43,7800.0,2.0,7800.0,0.0,0.0,0.0,0.0,19.220107,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
349,0.0,33,7955.0,3.0,0.0,0.0,1455.0,4150.0,2350.0,0.262839,1,0
350,0.0,54,10320.0,2.0,1950.0,8370.0,0.0,0.0,0.0,0.229984,1,0
351,1.0,39,2182.0,1.0,0.0,0.0,2182.0,0.0,0.0,1.839874,1,0
352,1.0,56,3727.0,1.0,0.0,0.0,0.0,3727.0,0.0,20.797142,1,0


### Data Split

In [410]:
# sklearn에서 제공하는 split 함수를 이용해보자. : train_test_split
from sklearn.model_selection import train_test_split

# features와 target 분리
X = customer_churn.drop('Churn', axis=1)
y = customer_churn.loc[:, 'Churn'] 

train_features, test_features, train_target, test_target \
    = train_test_split(X, y, test_size=0.2, random_state=1)

### 문제 7. Scalinng

숫자형 변수들을 스케일링 합니다.

In [411]:
# 필요한 함수 로딩
from sklearn.preprocessing import MinMaxScaler

# 선언하기
minmax = MinMaxScaler()

# train 데이터로 .fit ==> 모든변수의 min, max, 공식 도출.
# 말하자면, scaling 하는 모델을 만드는 것!
minmax.fit(train_features) # train min, max

# 위 scaling 공식으로 적용하기(변환하기)
train_features_sc = minmax.transform(train_features)
test_features_sc = minmax.transform(test_features)

In [412]:
train_target, test_target = train_target.values, test_target.values

### 모델링
다음 코드는 그대로 실행하여 결과를 확인해 봅시다.

In [413]:
# 필요한 라이브러리 가져오자
from sklearn.neighbors import KNeighborsClassifier

# confusion matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# 모델 준비
knn_cl = KNeighborsClassifier(n_neighbors=3)
knn_cl.fit(train_features, train_target)
test_pred = knn_cl.predict(test_features)

test_acc = accuracy_score(test_target, test_pred)
print('test 정확도 : {:.2f}%'.format(test_acc*100))


test 정확도 : 66.20%


정 분류율(Accuracy)은 몇 % 인가요?  

In [414]:
print(confusion_matrix(test_target, test_pred))

[[31 12]
 [12 16]]


### 문제 8. 성능을 더 높이기 위해서 무엇을 하면 좋을까요?

여러분의 의견을 아래 적어 봅시다.